In [59]:
import psycopg2
import getpass
import pandas as pd
import datetime




In [8]:
# коннектимся к базе данных
conn=psycopg2.connect(user='postgres',
                      database='postgres',
                      host='localhost',
                      port='5432',
                      password=getpass.getpass('Insert password: ')
                        )

Insert password: ········


In [135]:
# запрос для необходимых данных. Т.к в данных по облигациям с фиксированным купоном встречаются разные купонные выплаты,
# сделано предположение использовать максимальный купонный платеж из имеющихся по облигации
query = """SELECT *
            FROM (SELECT MAX("CPN") AS "CPN", "ISIN", max("BUYBACKDATE") AS "BUYBACKDATE",
                  max("BUYBACKPRICE") AS "BUYBACKPRICE"
                  FROM bonds.quotes  
                  GROUP BY "ISIN") AS cpn
            INNER JOIN(SELECT "ISINCode", "FaceValue", "HaveOffer", "AmortisedMty", 
                       "CouponPerYear", "NumCoupons", "EndMtyDate", "CouponType", "IssuerName"
                       FROM bonds.bond_description 
                       WHERE "Status"!='Погашен' AND "CouponType" != 'Переменный' ) AS bonds
            ON cpn."ISIN" = bonds."ISINCode" """

In [136]:
# забираем нужные данные из базы
data = pd.read_sql_query(query, conn)
data = pd.DataFrame(data)
data.head()

,CPN,ISIN,BUYBACKDATE,BUYBACKPRICE,ISINCode,FaceValue,HaveOffer,AmortisedMty,CouponPerYear,NumCoupons,EndMtyDate,CouponType,IssuerName
0,NaN,RU000A0JNN80,None,0.0,RU000A0JNN80,1000.0,False,False,2,6,2009-09-17,Постоянный,МГ Групп
1,40.11,RU000A0JNYN1,None,NaN,RU000A0JNYN1,1000.0,False,False,2,31,2022-06-11,Фиксированный,Москва Правительство
2,40.11,RU000A0JNYP6,None,0.0,RU000A0JNYP6,1000.0,True,False,2,21,2017-06-14,Фиксированный,Москва Правительство
3,40.33,RU000A0JPZ50,None,NaN,RU000A0JPZ50,1000.0,True,False,2,16,2016-09-22,Фиксированный,Москва Правительство
4,40.33,RU000A0JPZ68,None,NaN,RU000A0JPZ68,1000.0,False,False,2,22,2019-09-29,Фиксированный,Москва Правительство


In [137]:
# убираем ISINCode, т.к. он требовался только для соединения таблиц, далее не нужен.
data = data.drop(['ISINCode'], axis=1)

data.head(5)

,CPN,ISIN,BUYBACKDATE,BUYBACKPRICE,FaceValue,HaveOffer,AmortisedMty,CouponPerYear,NumCoupons,EndMtyDate,CouponType,IssuerName
0,NaN,RU000A0JNN80,None,0.0,1000.0,False,False,2,6,2009-09-17,Постоянный,МГ Групп
1,40.11,RU000A0JNYN1,None,NaN,1000.0,False,False,2,31,2022-06-11,Фиксированный,Москва Правительство
2,40.11,RU000A0JNYP6,None,0.0,1000.0,True,False,2,21,2017-06-14,Фиксированный,Москва Правительство
3,40.33,RU000A0JPZ50,None,NaN,1000.0,True,False,2,16,2016-09-22,Фиксированный,Москва Правительство
4,40.33,RU000A0JPZ68,None,NaN,1000.0,False,False,2,22,2019-09-29,Фиксированный,Москва Правительство


In [142]:
def get_date(date_entry):
    # метод для получения даты из строки
    # формат YYYY-MM-DD
    year, month, day = map(int, date_entry.split('-'))
    return datetime.date(year, month, day)

In [143]:
def build_matrix_dict(issuer_name, terminal_date):
    # составим список дат платежей для каждой облигации
    
    # получим нужные облигации
    terminal_date = get_date(terminal_date)
    obligations = data[data['IssuerName'] == issuer_name]
    
    
    matrix = dict()
    
    # для каждой облигации составляем список дат
    for index, obligation in obligations.iterrows():
        matrix[obligation['ISIN']] = []
        
        # считаем периодичность в зависимости от количества выплат в течение года (в дняхх)
        periodicity = 365 // obligation['CouponPerYear']
        
        # учитываем дату отсчета
        payment_date = None
        if obligation['BUYBACKDATE'] is not None:
            payment_date = obligation['BUYBACKDATE']
        else:
            payment_date = obligation['EndMtyDate']
            
        for i in range(obligation['NumCoupons']):
            
            # даты больше введеной пропускаем
            if payment_date > terminal_date:
                pass
            else:
                matrix[obligation['ISIN']].append(payment_date)
            
            # считаем следующую дату выплаты
            payment_date = payment_date - datetime.timedelta(days=periodicity)
    return matrix    
    

In [139]:
def build_matrix_df(matrix_dict):
    # строим дата фрейм для матрицы дат
    
    # сначала получим список всех дат всех облигаций, они будут колонками
    date_set=set()
    for isin in matrix_dict:
        date_set.update(matrix_dict[isin])
    date_set = list(date_set)
    date_set.sort()
    
    # строим дата фрейм, где строки - облигации, колонки - даты
    I = pd.Index(matrix_dict.keys(), name="rows")
    C = pd.Index(date_set, name="cols")
    df = pd.DataFrame(data='-', index = I, columns = C)
    
    # заполняем дата фрейм
    for isin in matrix_dict:
        
        # для каждой облигации берем список дат
        datelist = matrix_dict[isin]
        
        # и заполняем его верным значением в зависимости от даты
        obligation = data[data['ISIN'] == isin].iloc[0]
        cpn = obligation['CPN']
        face_value = obligation['FaceValue']
        
        # определяем верную дату
        terminal_date = None
        if obligation['BUYBACKDATE'] is not None:
            terminal_date = obligation['BUYBACKDATE']
        else:
            terminal_date = obligation['EndMtyDate']
            
        for the_date in datelist:
        # и наконец заполняем значение выплаты
            if the_date == terminal_date:
                df.at[isin, the_date] = cpn + face_value
            else:
                df.at[isin, the_date] = cpn
    return df

In [141]:
issuer_name = input('Введите IssuerName')
term_date = input('Введите дату в формате YYYY-MM-DD')
matrix = build_matrix_dict(issuer_name, term_date)
build_matrix_df(matrix).head(15)

Введите IssuerNameБанк ВТБ
Введите дату в формате YYYY-MM-DD2018-06-06


cols,2013-04-23,2013-05-21,2013-07-23,2013-08-20,2013-10-22,2013-11-19,2014-01-21,2014-02-18,2014-04-22,2014-05-20,...,2015-01-20,2015-02-17,2015-04-21,2015-05-19,2015-07-21,2015-08-18,2015-10-20,2015-11-17,2016-01-19,2016-02-16
rows,,,,,,,,,,,,,,,,,,,,,
RU000A0JTK20,20.32,-,20.32,-,20.32,-,20.32,-,20.32,-,...,20.32,-,20.32,-,20.32,-,20.32,-,1020.32,-
RU000A0JTN01,-,19.7,-,19.7,-,19.7,-,19.7,-,19.7,...,-,19.7,-,19.7,-,19.7,-,19.7,-,1019.7
